In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from scipy.spatial import distance

import tensorflow as tf
from keras import Sequential
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
train_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Train'
test_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Test'
val_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Validation'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255, horizontal_flip=True, zoom_range=0.2,shear_range=0.2, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(directory=train_dir,target_size=(128,128),class_mode='categorical',
                                                    batch_size=32,
                                                    subset='training')

val_generator = train_datagen.flow_from_directory(directory=train_dir,target_size=(128,128),class_mode='categorical',
                                                subset='validation',
                                                batch_size=32)

test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = train_datagen.flow_from_directory(directory=val_dir,target_size=(128,128),batch_size=32)

In [2]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(128, 128, 3),
                                               include_top=False,
                                               weights='imagenet')

for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    Dense(2, activation='sigmoid'),
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_128 (Funct  (None, 4, 4, 1280)       2257984   
 ional)                                                          
                                                                 
 flatten (Flatten)           (None, 20480)             0         
                                                                 
 dense (Dense)               (None, 2)                 40962     
                                                                 
Total params: 2,298,946
Trainable params: 40,962
Non-trainable params: 2,257,984
_________________________________________________________________


In [3]:
tf.keras.utils.plot_model(model, to_file="masknet.jpg", show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [5]:
early_stopping_callback = EarlyStopping(monitor='val_loss',
                                            patience=10,
                                            mode='min',
                                            restore_best_weights=True)

In [6]:
model.compile(optimizer="adam",loss="binary_crossentropy", metrics ="binary_accuracy")

In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch=len(train_generator)//32,
                    epochs=20,
                    validation_data=val_generator,
                    verbose=1,
                   callbacks=[early_stopping_callback])

In [ ]:
acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs,acc)
plt.plot(epochs,val_acc)

plt.figure()

plt.plot(epochs,loss)
plt.plot(epochs,val_loss)

In [ ]:
model.evaluate_generator(test_generator)

In [ ]:
model.save('masknet.h5')